# Imports & Settings


In [1]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from deep_translator import GoogleTranslator
import re 
from math import isnan

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

In [19]:
# dtype={'type': str} prevents being confused with data type for large data sets
train = pd.read_csv('data/train.csv', index_col='id', dtype={'type': str})
test = pd.read_csv('data/test.csv', index_col='id', dtype={'type': str})
train_translated = pd.read_csv('data/train_translated.csv', dtype={'type': str})
test_translated = pd.read_csv('data/test_translated.csv', index_col='id', dtype={'type': str})
combined_data = pd.read_csv('data/combined_data.csv', index_col='id', dtype={'type': str})
combined_data_translated = pd.read_csv('data/combined_data_translated.csv', index_col='id', dtype={'type': str})
combined_data_fully_translated = pd.read_csv('data/combined_data_fully_translated.csv', index_col='id', dtype={'type': str})
prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})

# Delete unneeded features


In [9]:
data = prep.copy()

In [13]:
data.drop(columns=['full_nr', 'name','commentary','country_and_unit','text','parameter','unit','value','legend','initial_info','additional_text'], inplace=True)

In [15]:
data.to_csv('data/prep.csv')

In [11]:
data = prep.copy()

In [12]:
data.drop(columns=['film frame IN mm²', 'photo format IN mm²', 'negative format IN mm²', 'text_features'], inplace=True)

In [13]:
data.to_csv('data/prep.csv')

# rename for xgboost

In [7]:
data = prep.copy()

In [4]:
data.rename(columns={'location_määramata [Narva-Jõesuu]':'location_määramata (Narva-Jõesuu)', 'location_määramata [Tallinn]':'location_määramata (Tallinn)', 'location_määramata [Tartu]':'location_määramata (Tartu)', 'location_määramata [Raplamaa]':'location_määramata (Raplamaa)'}, inplace=True)

In [5]:
data.to_csv('data/prep.csv')

# delete too sparse one-hot endcoded features

In [3]:
data = prep.copy()

In [4]:
import helpers

In [17]:
tech = helpers.col_collection(data, 'technique_')
mat = helpers.col_collection(data, 'material_')
loc = helpers.col_collection(data, 'location_')

80 columns found that start with technique_
80 columns found that start with material_
149 columns found that start with location_


In [7]:
cols = tech + mat + loc


In [15]:
too_few = []

for col in cols:
    if data[col].sum() < 2:
        too_few.append(col)

In [16]:
data.drop(columns=too_few, inplace=True)

In [18]:
data.to_csv('data/prep.csv')

# resplit test/train


In [20]:
data = prep.copy()

In [21]:
train = data.loc[data['source']=='train'].drop('source',axis=1)
test = data.loc[data['source']=='test'].drop('source',axis=1)

In [22]:
train.to_csv('data/train_prepared.csv')
test.to_csv('data/test_prepared.csv')